In [21]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys

In [63]:
def prepare_data(title, label_map_filename, ext='.npz'):
    action_classes = [x.strip() for x in open(label_map_filename)]
    res = np.load(title + ext)
    preds = res['out_logits']
    truth_labels_txt = res['truth_labels']
    truth_labels = np.zeros([truth_labels_txt.shape[0],1])
    
    for i in range(truth_labels.shape[0]):
        truth_labels[i] = int( action_classes.index(truth_labels_txt[i]) )
        
    return action_classes, truth_labels, preds
    


def topN_acc(preds, truth_labels, n=1):
    top_preds = np.argsort(preds, 1)
    topN_preds = top_preds[:,-n:]
    is_topN = np.zeros(preds.shape[0])
    for i in range(preds.shape[0]):
        is_topN[i] = np.isin(truth_labels[i], topN_preds[i,:])
        
    return np.sum(is_topN,0) / is_topN.shape[0]

def confusion_matrix(preds, truth_labels):
    #num_classes = len(action_classes)
    num_classes = preds.shape[1]
    conf_matrix = np.zeros([num_classes,num_classes])
    top_preds = np.argmax(preds, 1)
    for i in range(truth_labels.shape[0]):
        conf_matrix[int(truth_labels[i,0]),top_preds[i]] += 1
    
    return conf_matrix

def summary_confmat(conf_mat, truth_labels, k=10):
    _summary_confmat = []
    for x in np.unique(truth_labels).astype(int):
        sorted_args = np.flip( np.argsort(conf_matrix_rgb[x,:])[-10:],0 )
        sorted_values = np.flip( np.sort(conf_matrix_rgb[x,:])[-10:],0 )
        _summary_confmat.append([action_classes[x], [np.take(action_classes, sorted_args), sorted_values]])
    return _summary_confmat

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True,
                          highlight_classes=[]):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(16, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",
                     fontweight="heavy" if (i == j and i in highlight_classes) else "normal")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",
                     fontweight="heavy" if (i == j and i in highlight_classes) else "normal")

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label)
    fig1 = plt.gcf()
    plt.show()
    fig1.savefig(title + '.pdf')
    
    
def merge_interrupted_results(title1, title2, first_run_last_iter, out_title, ext='.npz'):
    res1 = np.load(title1 + ext)
    res2 = np.load(title2 + ext)    
    preds1 = res1['out_logits']
    preds2 = res2['out_logits']
    truth_labels = res1['truth_labels']
    if first_run_last_iter == None:
        for i in range(preds1.shape[0]):
            if np.sum(preds1[i,:]) == 0:
                first_run_last_iter = i

    preds = np.zeros(preds2.shape)
    preds[0:first_run_last_iter,:] = preds1[0:first_run_last_iter,:]
    preds[first_run_last_iter+1:-1,:] = preds2[first_run_last_iter+1:-1,:]
    
    np.savez(out_title + ext, truth_labels=truth_labels, out_logits=preds)
    
    
def produce_results(title, label_map_filename, ext='.npz'):
    #Getting action classes, truth labels and single stream predictions
    action_classes, truth_labels, preds = prepare_data(title,label_map_filename, ext='.npz')
    
    # getting classification accuracies and saving to text files
    top1_acc = topN_acc(preds, truth_labels,n=1)
    top5_acc = topN_acc(preds, truth_labels,n=5)
    top10_acc = topN_acc(preds, truth_labels,n=10)
    print("top1_acc: " + str(top1_acc))
    print("top5_acc: " + str(top5_acc))
    print("top10_acc: " + str(top10_acc))

    np.savetxt(title + '.txt',(top1_acc, top5_acc, top10_acc) )
    
    # Creating confusion matrix for entire set of action classses
    conf_matrix = confusion_matrix(preds, truth_labels)
    
    # Creating smaller confusion matrix for evaluated classes
    kept_action_indices = np.unique(truth_labels).astype(int)
    classes_array = np.asarray(action_classes)
    kept_actions = classes_array[kept_action_indices]
    small_confmat = conf_matrix[kept_action_indices,:]
    small_confmat = small_confmat[:, kept_action_indices]
    
    # Rewriting some action labels for clarclarityify on confusion matrix
    kept_actions[5] = 'soccer_juggling'
    kept_actions[8] = 'squash'
    
    # Rearrange confusion matrix for better clarity
    kept_actions_tmp = kept_actions
    small_confmat_tmp = small_confmat
    
    kept_actions_tmp[0] = "punching_person"
    kept_actions_tmp[1] = "side_kick"
    kept_actions_tmp[2] = "high_kick"
    kept_actions_tmp[3] = "slapping"
    kept_actions_tmp[4] = "wrestling"
    kept_actions_tmp[5] = "drop_kicking"
    kept_actions_tmp[6] = "sword_fighting"
    kept_actions_tmp[7] = "headbutting"
    
    kept_actions_tmp[8] = "punching_bag"
    kept_actions_tmp[9] = "singing"
    kept_actions_tmp[10] = "playing_squash_or_racquetball"
    kept_actions_tmp[11] = "stretching_arm"
    kept_actions_tmp[12] = "capoeira"
    kept_actions_tmp[13] = "juggling_soccer_ball"
    kept_actions_tmp[14] = "playing_cricket"
    kept_actions_tmp[15] = "kissing"
    kept_actions_tmp[16] = "shaking_head"
    kept_actions_tmp[17] = "headbanging"
    kept_actions_tmp[18] = "tai_chi"
    kept_actions_tmp[19] = "tango_dancing"
    
    kept_action_indices = np.ones(20)
    for i in range(20):
        kept_action_indices[i] = int(action_classes.index(kept_actions_tmp[i]))
        
    print (kept_action_indices)
    small_confmat = conf_matrix[kept_action_indices.astype(int),:]
    small_confmat = small_confmat[:, kept_action_indices.astype(int)]
    
    kept_actions_tmp[10] = "playing_squash"
    kept_actions_tmp[13] = "soccer_juggling"
    
    kept_actions = kept_actions_tmp
    
    
    #plotting and saving confusion matrix as pdf image
    plot_confusion_matrix(cm           = small_confmat, 
                          normalize    = False,
                          target_names = kept_actions,
                          title        = title,
                          highlight_classes = [0,1,2,3,4,5,6,7])
    
def compute_joint(rgb_logits, flow_logits):
    out_logit = torch.from_numpy(rgb_logits) + torch.from_numpy(flow_logits)
    out_softmax = torch.nn.functional.softmax(out_logit, 1).data.cpu()
 
    '''
    top_val, top_idx = torch.sort(out_softmax, 1, descending=True)
    print('===== Final predictions ====')
    print('logits proba class '.format(args.top_k))
    for i in range(args.top_k):
        logit_score = out_logit[0, top_idx[0, i]].data[0]
        print('{:.6e} {:.6e} {}'.format(logit_score, top_val[0, i],
                                        kinetics_classes[top_idx[0, i]]))
    '''
    return out_logit, out_softmax


SyntaxError: EOL while scanning string literal (<ipython-input-63-fd33afcf07df>, line 122)

In [9]:
label_map_filename = 'label_map.txt'
ext = '.npz'

action_classes, truth_labels, logits_rgb = prepare_data('i3d-ViolentHumanActions_v2-rgb', label_map_filename,  ext='.npz')
action_classes, truth_labels, logits_flow = prepare_data('i3d-ViolentHumanActions_v2-flow',label_map_filename, ext='.npz')

logits_joint, softmax_joint = compute_joint(logits_rgb, logits_flow)

In [26]:
label_map_filename = 'label_map.txt'
ext = '.npz'
title_softmax_joint = 'i3d-ViolentHumanActions_v2-softmax_joint'
title_softmax_average = 'i3d-ViolentHumanActions_v2-softmax_average'
title_logits_joint = 'i3d-ViolentHumanActions_v2-logits_joint'

title = 'i3d-ViolentHumanActions_v2-rgb'
res = np.load(title + ext)
truth_labels_txt = res['truth_labels']


res = np.load(title_softmax_joint + ext)
preds = res['out_logits']
np.savez(title_softmax_joint + ext, truth_labels=truth_labels_txt, out_logits=preds)

res = np.load(title_softmax_average + ext)
preds = res['out_logits']
np.savez(title_softmax_average + ext, truth_labels=truth_labels_txt, out_logits=preds)

res = np.load(title_logits_joint + ext)
preds = res['out_logits']
np.savez(title_logits_joint + ext, truth_labels=truth_labels_txt, out_logits=preds)



In [42]:
title = 'i3d-ViolentHumanActions_v2-flow-COPY'
res = np.load(title + ext)
preds = res['out_logits']
truth_labels = res['truth_labels']

preds[:,action_classes.index('punching_person')] = -100

np.savez(title + ext, truth_labels=truth_labels, out_logits=preds)

In [ ]:
label_map_filename = 'label_map.txt'
ext = '.npz'

titles = []


titles.append('i3d-ViolentHumanActions_v1-flow')
titles.append('i3d-ViolentHumanActions_v1-rgb')
titles.append('i3d-ViolentHumanActions_v2_15fps-flow')
titles.append('i3d-ViolentHumanActions_v2_15fps-rgb')
titles.append('i3d-ViolentHumanActions_v2-flow')
titles.append('i3d-ViolentHumanActions_v2-rgb')
titles.append('i3d-ViolentHumanActions_v2-softmax_joint')
titles.append('i3d-ViolentHumanActions_v2-softmax_average')
titles.append('i3d-ViolentHumanActions_v2-logits_joint')


#titles.append('i3d-ViolentHumanActions_v2-flow')
#titles.append('i3d-ViolentHumanActions_v2-flow-COPY')

ext = '.npz'

for title in titles:
    produce_results(title, label_map_filename, ext)

In [32]:
i = 5
a = [1,5,3]
i in a

True